## Requirements


In [1]:
# Unable warnings
import os
import warnings

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#### Directory adjustment


In [2]:
from pathlib import Path
import sys
import os

# Back to main folder
path = os.path.dirname(os.getcwd()) + "/"
os.chdir(path)
sys.path.append(path)


#### Data Processing


In [3]:
# ETL
import numpy as np
import pandas as pd
import multiprocessing

# ML preprocessing
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.feature_selection import chi2


#### Natural language processing


In [4]:
# Basic
import re

# Tracking
from src.experiment.tracking import experiment

# Vectorization
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from src.transformers.text import TextNormalizer

# Embedding
from src.transformers.vectorizer import MeanEmbeddingVectorizer
from gensim.models import KeyedVectors
from gensim import models
import gensim


#### Models


In [5]:
# Pipe
from sklearn.pipeline import Pipeline

# Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

# Metrics
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score


### DVC


In [6]:
from src.data.control import version

df_train, df_test = version().split(test_size=0.2)


## Data manipulation


In [7]:
# Set target and features
target = "label"
features = "text"

# Set train and test
X_train, y_train = df_train[features], df_train[target]
X_test, y_test = df_test[features], df_test[target]


# Set k-fold criteria
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)


# Class weights
pos = len(df_train.query("label==1"))
neg = len(df_train.query("label==0"))
weight_for_0 = (1 / neg) * (len(df_train) / 2.0)
weight_for_1 = (1 / pos) * (len(df_train) / 2.0) 
relative_weight = {0: weight_for_0, 1: weight_for_1}


## Model steps

In [8]:
# Text normalizer
wordlist = [
    "nomeusuario",
    "paginaweb",
    "emailusario",
    "numerotelefone",
    "simbolomonetario",
]

normalizer = TextNormalizer(
    stopwords=True, wordlist=wordlist, stemmer=False, lemma=False
)


# Classifiers
classifiers = {
    "GradientBoosting": GradientBoostingClassifier(),
    "KNeighbors": KNeighborsClassifier(),
    "Bernoulli": BernoulliNB(alpha=1.0, binarize=0.0, fit_prior=True, class_prior=None),
    "SVC": LinearSVC(
        penalty="l2",
        loss="squared_hinge",
        dual=True,
        tol=1e-6,
        C=1.1,
        multi_class="crammer_singer",
        fit_intercept=True,
        intercept_scaling=1,
        class_weight=relative_weight,
        random_state=42,
        max_iter=1000,
    ),
    "LogisticRegression": LogisticRegression(penalty="l2", max_iter=200, C=1),
    "SGDC": SGDClassifier(loss="hinge", max_iter=200),
    "DecisionTree": DecisionTreeClassifier(
        random_state=42,
        class_weight=relative_weight,
        min_samples_split=2,
        max_features="auto",
    ),
    "RandomForest": RandomForestClassifier(
        random_state=42,
        class_weight=relative_weight,
        min_samples_split=2,
        max_features="auto",
        oob_score=True,
    ),
    "SVM": svm.SVC(kernel="rbf"),
}


## My own word2vec embedding model

In [9]:
# Define a corpus
corpus = X_train
cores = multiprocessing.cpu_count()

# Train a own word2vec model
my_embedding_50d = gensim.models.Word2Vec(
    corpus,
    vector_size=50,
    window=4,
    min_count=10,
    sg=1,
    workers=cores - 1,
    batch_words=10000,
    alpha=0.1,
    min_alpha=0.0001,
    negative=20,
)

my_embedding_100d = gensim.models.Word2Vec(
    corpus,
    vector_size=50,
    window=4,
    min_count=10,
    sg=1,
    workers=cores - 1,
    batch_words=10000,
    alpha=0.1,
    min_alpha=0.0001,
    negative=20,
)

my_embedding_300d = gensim.models.Word2Vec(
    corpus,
    vector_size=300,
    window=4,
    min_count=10,
    sg=1,
    workers=cores - 1,
    batch_words=10000,
    alpha=0.1,
    min_alpha=0.0001,
    negative=20,
)


my_embedding_1000d = gensim.models.Word2Vec(
    corpus,
    vector_size=1000,
    window=4,
    min_count=10,
    sg=1,
    workers=cores - 1,
    batch_words=10000,
    alpha=0.1,
    min_alpha=0.0001,
    negative=20,
)

# Make embedding dictionary {token:vector}
my_embedding_50d = dict(
    zip(my_embedding_50d.wv.index_to_key, my_embedding_50d.wv.vectors)
)

my_embedding_100d = dict(
    zip(my_embedding_100d.wv.index_to_key, my_embedding_100d.wv.vectors)
)

my_embedding_300d = dict(
    zip(my_embedding_300d.wv.index_to_key, my_embedding_300d.wv.vectors)
)

my_embedding_1000d = dict(
    zip(my_embedding_1000d.wv.index_to_key, my_embedding_1000d.wv.vectors)
)


# Make a vectorizer
vec_50 = MeanEmbeddingVectorizer(my_embedding_50d)
vec_100 = MeanEmbeddingVectorizer(my_embedding_100d)
vec_300 = MeanEmbeddingVectorizer(my_embedding_300d)
vec_1000 = MeanEmbeddingVectorizer(my_embedding_1000d)

# List embeddings
embeddings = {
    "w2v_50": vec_50,
    "w2v_100": vec_100,
    "w2v_300": vec_300,
    "w2v_1000": vec_1000,
}


### Basic pipeline

In [10]:
# Basic pipeline
ml_pipe = Pipeline(
    [
        ("normalizer", normalizer),
        ("vectorizer", vec_50),
        ("classifier", classifiers["SVC"]),
    ]
)

# Train
ml_pipe.fit(X_train, y_train)

pd.DataFrame(classification_report(y_test, ml_pipe.predict(X_test), output_dict=True))


,0,1,accuracy,macro avg,weighted avg
precision,0.767318,0.395797,0.580247,0.581557,0.650358
recall,0.555985,0.633053,0.580247,0.594519,0.580247
f1-score,0.644776,0.487069,0.580247,0.565923,0.595128
support,777.000000,357.000000,0.580247,1134.000000,1134.000000


### Multi model x Multi Embedding

In [11]:
for model_name, classifier in classifiers.items():
    for embedding_name, embedding in embeddings.items():

        # Build a classifier pipeline
        ml_pipe = Pipeline(
            [
                ("normalizer", normalizer),
                ("vectorizer", embedding),
                ("classifier", classifier),
            ]
        )

        # Set experiment
        lab = experiment(
            exp_name="Hate Speech",
            model_name=f"{model_name} - {embedding_name}",
            model=ml_pipe,
        )

        # Evaluate experiment
        y_pred = lab.run(X_train, y_train, X_test, y_test)


[MLFLOW] [START] starting server


[2022-11-03 10:51:36 -0300] [46766] [INFO] Starting gunicorn 20.1.0
[2022-11-03 10:51:36 -0300] [46766] [INFO] Listening at: http://127.0.0.1:7500 (46766)
[2022-11-03 10:51:36 -0300] [46766] [INFO] Using worker: sync
[2022-11-03 10:51:36 -0300] [46777] [INFO] Booting worker with pid: 46777
[2022-11-03 10:51:36 -0300] [46778] [INFO] Booting worker with pid: 46778
[2022-11-03 10:51:36 -0300] [46779] [INFO] Booting worker with pid: 46779
[2022-11-03 10:51:36 -0300] [46780] [INFO] Booting worker with pid: 46780


[MLFLOW][EXECUTION] running experiment


2022/11/03 10:51:59 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2022-09-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'minhasoma'}
2022/11/03 10:52:04 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2022-09-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'minhasoma'}


[MLFLOW] [FINISHED] experiment executed successfully
model:GradientBoosting - w2v_50 - acc:0.6948853615520282 - rec:0.12605042016806722 - auc:0.5411461882050118 - f1:0.20642201834862386 

[MLFLOW] [START] server already running
[MLFLOW][EXECUTION] running experiment


...

[MLFLOW] [FINISHED] experiment executed successfully
model:SVM - w2v_100 - acc:0.6851851851851852 - rec:0.0 - auc:0.5 - f1:0.0 

[MLFLOW] [START] server already running
[MLFLOW][EXECUTION] running experiment


2022/11/03 11:05:51 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2022-09-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'minhasoma'}
2022/11/03 11:05:56 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2022-09-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'minhasoma'}


[MLFLOW] [FINISHED] experiment executed successfully
model:SVM - w2v_300 - acc:0.6851851851851852 - rec:0.0 - auc:0.5 - f1:0.0 

[MLFLOW] [START] server already running
[MLFLOW][EXECUTION] running experiment


2022/11/03 11:07:00 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2022-09-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'minhasoma'}
2022/11/03 11:07:07 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2022-09-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'minhasoma'}


[MLFLOW] [FINISHED] experiment executed successfully
model:SVM - w2v_1000 - acc:0.6851851851851852 - rec:0.0 - auc:0.5 - f1:0.0 



## Pre-trained embeddings


In [14]:
# Load a pre-trained model
pretrained_model = models.KeyedVectors.load_word2vec_format(
    "data/pretrained-skipgram/skip_s50.txt", binary=False
)

# Make embedding dictionary {token:vector}
pre_embedding_50d = dict(zip(pretrained_model.index_to_key, pretrained_model.vectors))

# Make a vectorizer
pre_vec_50 = MeanEmbeddingVectorizer(pre_embedding_50d)


### Basic pipeline

In [15]:
# Basic pipeline
ml_pipe = Pipeline(
    [
        ("normalizer", normalizer),
        ("vectorizer", pre_vec_50),
        ("classifier", classifiers["SVC"]),
    ]
)

# Train
ml_pipe.fit(X_train, y_train)

pd.DataFrame(classification_report(y_test, ml_pipe.predict(X_test), output_dict=True))


2022/11/03 11:08:35 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e76583d22cc1406698bae430bd6e0c7a', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/11/03 11:08:55 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2022-09-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'minhasoma'}


,0,1,accuracy,macro avg,weighted avg
precision,0.760902,0.422175,0.620811,0.591539,0.654266
recall,0.651223,0.554622,0.620811,0.602922,0.620811
f1-score,0.701803,0.479419,0.620811,0.590611,0.631793
support,777.000000,357.000000,0.620811,1134.000000,1134.000000


### Multi model x Multi embedding
Run `src/TrainMultiModel.py`